In [81]:
import numpy as np
import pandas as pd

In [93]:
df = pd.read_excel("Kollektorvæskedata.xlsx", sheet_name="Sheet1")
vaeske = 'Ethanole 2, 4 %'
fraksjon = 0.24
vaesketemp = 0

riktig_rad = df[df['Cooling liquid'].str.contains(vaeske)] 
riktig_rad = riktig_rad.loc[riktig_rad['fraction']==fraksjon]
riktig_rad = riktig_rad.loc[riktig_rad['temp (C)']==vaesketemp]

print(riktig_rad)

k = riktig_rad.iloc[0,3]
cp = riktig_rad.iloc[0,4]
tetthet = riktig_rad.iloc[0,5]
viskositet = riktig_rad.iloc[0,6]
Pr = riktig_rad.iloc[0,8]

     Cooling liquid  fraction  temp (C)      k      cp   dens     visc  \
41  Ethanole 2, 4 %      0.24       0.0  0.428  4295.3  972.3  0.00579   

    fryspunk         Pr  
41     -14.6  58.106979  
58.106978971962626


In [67]:
antall_bronner = 9
volstrom = (150/3.6)/60
ruhet = 0.0000015

In [68]:
rordata = np.array([[32,2.0],[40,2.4],[50,3.0],[63,3.8],[75,4.5],[90,5.4],[110,6.6],[125,7.4],[140,8.3],[160,9.5],[180,10.7],[200,11.9],[225,13.4],[250,14.8],[280,16.6],
                    [315,18.7],[355,21.1],[400,23.7],[450,26.7],[500,29.7],[560,33.2],[600,35.6],[630,37.4],[710,42.1],[800,47.4],[900,53.3],[1000,59.3],[1100,65.2],[1200,70.6],
                    [1400,82.4],[1600,94.1],[1800,105.9],[2000,117.6]])

In [69]:
def trykktapsfunksjon(volstrom,tetthet,diam_ytre,veggtykkelse,viskositet,ruhet,lengde):
    massestrom = volstrom*tetthet/1000
    diam_indre = (diam_ytre-2*veggtykkelse)/1000
    Re = (4*massestrom)/(viskositet*np.pi*diam_indre)
    frikfaktor = (1/(-1.8*np.log10(6.9/Re)+((ruhet/diam_indre)/3.7)**1.11))**2
    hast = massestrom/(tetthet*np.pi*(diam_indre/2)**2)
    trykktap = frikfaktor*lengde/diam_indre*(tetthet*hast**2)/2
    return Re,trykktap

def trykktapsfunksjon_VP(volstrom,tetthet,diam_ytre,veggtykkelse,viskositet,lengde):
    massestrom = volstrom*tetthet/1000
    diam_indre = (diam_ytre-2*veggtykkelse)/1000
    Re = (4*massestrom)/(viskositet*np.pi*diam_indre)
    frikfaktor = 64/Re
    hast = massestrom/(tetthet*np.pi*(diam_indre/2)**2)
    trykktap = frikfaktor*lengde/diam_indre*(tetthet*hast**2)/2
    return trykktap

In [80]:
trykktapsgrense = 100000        # Pa
lengde_bronn = 640
for i in range(0,len(rordata)):
    veggtykkelse_bronn = rordata[i,1]    # mm
    diam_ytre_bronn = rordata[i,0]        # mm
    [Re_bronn, trykktap_bronn] = trykktapsfunksjon(volstrom,tetthet,diam_ytre_bronn,veggtykkelse_bronn,viskositet,ruhet,lengde_bronn)
    if trykktap_bronn <= trykktapsgrense:
        print(diam_ytre_bronn)
        print(veggtykkelse_bronn)
        print(trykktap_bronn)
        break
    

564187.5736944089
193346.5007625544
68233.12677982128
50.0
3.0
68233.12677982128


In [71]:
lengde_trase = 110
veggtykkelse_trase = 2.4    # mm
diam_ytre_trase = 40        # mm

[Re_trase, trykktap_trase] = trykktapsfunksjon(volstrom,tetthet,diam_ytre_trase,veggtykkelse_trase,viskositet,ruhet,lengde_trase)

print(trykktap_trase)

33231.429818564044


In [72]:
lengde_ledning = 110
volstrom_ledning = antall_bronner*volstrom
veggtykkelse_ledning = 7.4    # mm
diam_ytre_ledning = 125        # mm

[Re_ledning, trykktap_ledning] = trykktapsfunksjon(volstrom,tetthet,diam_ytre_ledning,veggtykkelse_ledning,viskositet,ruhet,lengde_ledning)

print(trykktap_ledning)

166.83884351169075


In [73]:
trykktap_VP = trykktapsfunksjon_VP(volstrom_ledning,tetthet,diam_ytre_ledning,veggtykkelse_ledning,viskositet,lengde_ledning)

print(trykktap_VP)

1443.5107207544524


In [74]:
trykktap_tot = trykktap_bronn+trykktap_trase+trykktap_ledning+trykktap_VP

print(trykktap_tot)

34841.78405677859


In [75]:
def Nusselt(Re,Pr):
    teller = (1/(0.79*np.log(Re)-1.64)**2)/(8)*(Re-1000)*Pr
    nevner = 1+12.7*((1/(0.79*np.log(Re)-1.64)**2)/(8))**(1/2)*Pr**(2/3)-1
    Nu = teller/nevner                                                      #Kun hvis Re > 2300 
    return Nu

Nu = Nusselt(Re_bronn,Pr)
h = (Nu*k)/((diam_ytre_bronn-2*veggtykkelse_bronn)/1000)
R_konv = 1/(np.pi*((diam_ytre_bronn-2*veggtykkelse_bronn)/1000)*h)
R_ror = 1/(2*np.pi*0.42)*np.log(0.02/0.0176)

print(Nu)
print(h)
print(R_konv)

-69.26636955179227
-14.679069703292333
-0.012287289746385367
